<h3>Importing libraries

In [2]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h3>Reading List of Postcal Codes of Canada from Wiki

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

<h3> Removing first row with titles and assigning names to columns
    Removing rows Not assigned

In [9]:

df = df.iloc[1:]
#column_names = ['PostalCode''Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
df.columns = ['PostalCode','Borough', 'Neighborhood']
# Removing Borough with Not assgined value
df = df[df["Borough"] != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue


 <h3> Removing "Not assigned" values on neighborhoods and assigning borough values to Neighborhood

In [10]:
for i in range(0, len(df)):
    
    for j in range(i+1, len(df)):
        #print("df.iloc[",i,"0]",df.iloc[i, 0],"  =  df.iloc[",j,", 0]",df.iloc[j, 0])
        if(j <= len(df)):
            #print("df.iloc[i, 0]",df.iloc[i, 0],"df.iloc[j, 0]",df.iloc[j, 0])
            if df.iloc[i, 2] == "Not assigned":
                
                #concatenate neighborhoods
                df.iloc[i, 2] = df.iloc[i, 1]
df.head(10)

,PostalCode,Borough,Neighborhood
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern
15,M3B,North York,Don Mills North
16,M4B,East York,Woodbine Gardens
17,M4B,East York,Parkview Hill


<h3> Finding same postal codes and concatenating text. First column (Postal code) is marked to later on be deleted

In [11]:
len1=len(df)
n=0

for i in range(0, len1):
    
    for j in range(i+1, len(df)):
        if(j <= len(df)):
            if df.iloc[i, 0] == df.iloc[j, 0]:
                #concatenate neighborhoods
                df.iloc[i, 2] = df.iloc[i, 2] + ', ' + df.iloc[j, 2]
                df.iloc[j, 0] = 'DEL'
                n=n+1            
#print ("n",n)
df.head(20)

len1  209
n 212


,PostalCode,Borough,Neighborhood
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,DEL,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,"Rouge, Malvern"
13,DEL,Scarborough,Malvern
15,M3B,North York,Don Mills North
16,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,DEL,East York,Parkview Hill


<h3> Deleting rows with a label DEL

In [12]:
df.columns = ['PostalCode','Borough', 'Neighborhood']
# Removing Borough with Not assgined value
df = df[df["PostalCode"] != 'DEL']
df.head(20)

,PostalCode,Borough,Neighborhood
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,"Lawrence Heights, Lawrence Manor"
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,"Rouge, Malvern"
15,M3B,North York,Don Mills North
16,M4B,East York,"Woodbine Gardens, Parkview Hill"
18,M5B,Downtown Toronto,"Ryerson, Garden District"
20,M6B,North York,Glencairn
23,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."


In [13]:
df.shape

(101, 3)